## What is Random Walk?

![RandomWalk_Image](../Documentation/RandomWalk.png)

## Deepwalk Methodology
DeepWalk is an approach used to represent the nodes of the Graph into a vector space of n-dimensions, such that they capture the social relations in the Graph. <br>
DeepWalk uses the local information of the network, by using the concept of Random Walk to traverse through the Graph.<br> In Random Walk, you start from a particular node and then choose one of the neighbours of that node randomly and add it to the path. From the Node Visited, you again choose one of its neighbours and add it to the path. This continues until the desired number of steps have been taken. Let’s understand this with an example.

![RandomWalk](../Documentation/RandomWalk-UserJourneyGraph.png)

Consider the Directed-Unweighted-Graph above. Say we start the Random Walk from Node A and the length of the walk is 4. From Node A, we can visit Node B. The path now becomes [A,B]. From Node B, we can choose to go to Node E or or even go back to Node A. Say, we choose to go to Node E. The path now becomes [A,B,E]. From Node E, we can choose to go to Node F or or Node C. Say, we choose to go to Node F then Random Walk <b>[A,B,E,F] of length 4</b> is generated.
<br>
As we can see, each Random Walk is a set of Nodes, comparing it to text, each Random Walk is like a sentence and each node is like a word in the sentence.

In DeepWalk, from each node, we generate Random Walks of length “l” a fixed number of times(“m”). This will generate number_of_nodes*m sequences of length “l”. We can then use the skip-gram approach to train a Word2Vec model.

<br>
<hr>

Reference: Word2Vec - [JayAlammar Word2vec Blog](https://jalammar.github.io/illustrated-word2vec/)

In [1]:
import pandas as pd
import numpy as np
from pecanpy import node2vec
from gensim.models import Word2Vec

from Constants import Constants
from tqdm import tqdm
from joblib import load, dump

GRAPH_FILE_NAME = Constants.GRAPH_FILE_NAME.value
WEIGHTED = Constants.WEIGHTED.value
DIRECTED = Constants.DIRECTED.value
NUM_WALK = Constants.NUM_WALK.value
WALK_LEN = Constants.WALK_LEN.value

print(GRAPH_FILE_NAME, NUM_WALK, WALK_LEN)

undirected_weighted_product_views_graph.parquet 10 50


### Let us consider undirected_weighted_product_views_graph to Generate RandomWalk Sequences

In [2]:
graph_path = '../Data/ConstructedGraph/{}'.format(GRAPH_FILE_NAME)
graph = pd.read_parquet(graph_path)

print("Graph Read from {}".format(graph_path))

Graph Read from ../Data/ConstructedGraph/undirected_weighted_product_views_graph.parquet


In [3]:
graph

,pid_1,pid_2,occurence_ct
0,3601512,3601269,37
1,100059274,3601306,12
2,5100855,4804056,880
3,100041977,100041934,2
4,100057579,100057560,11
...,...,...,...
6846587,12300752,4803895,1
6846588,100017063,6501098,1
6846589,8800741,1005195,1
6846590,25200369,23900098,1


In [4]:
Node_Count = len(set(graph['pid_1'].unique()).union(graph['pid_2'].unique()))
Edge_Count = len(graph)

print("Graph {} has Nodes: {} | Edges: {}".format(GRAPH_FILE_NAME, Node_Count, Edge_Count))

Graph undirected_weighted_product_views_graph.parquet has Nodes: 211861 | Edges: 6846592


#### Save Graph in .edg format

In [5]:
edg_graph_path = '../Data/Edg_Graphs_DataFile/'+GRAPH_FILE_NAME.split('.')[0]+'.edg'
graph.to_csv(edg_graph_path, sep='\t', index=False, header=False)
print("Edg Graph Saved")

Edg Graph Saved


### Graph Random Walk Generation

In [6]:
# Deepwalk has p=1 and q=1 (BFS and DFS equally important)
g = node2vec.SparseOTF(p=1, q=1, workers=-1, verbose=True, extend=False)

In [23]:
%%time
g.read_edg(edg_graph_path, weighted=WEIGHTED, directed=DIRECTED)

walks = g.simulate_walks(num_walks=NUM_WALK, walk_length=WALK_LEN)

Thread #  6 progress: |###                      | 9.99 %
Thread #  3 progress: |###                      | 9.99 %
Thread #  7 progress: |###                      | 9.99 %
Thread #  5 progress: |###                      | 9.99 %
Thread #  8 progress: |###                      | 9.99 %
Thread #  0 progress: |###                      | 9.99 %
Thread #  1 progress: |###                      | 9.99 %
Thread #  9 progress: |###                      | 9.99 %
Thread #  4 progress: |###                      | 9.99 %
Thread # 11 progress: |###                      | 9.99 %
Thread #  2 progress: |###                      | 9.99 %
Thread # 10 progress: |###                      | 9.99 %
Thread #  6 progress: |#####                    | 19.99 %
Thread #  3 progress: |#####                    | 19.99 %
Thread #  5 progress: |#####                    | 19.99 %
Thread #  7 progress: |#####                    | 19.99 %
Thread #  9 progress: |#####                    | 19.99 %
Thread #  1 progress: |###

In [24]:
assert len(walks) == Node_Count*NUM_WALK

In [18]:
print(walks[0])

['21405226', '21402696', '21403660', '21401562', '21401454', '21401223', '21400508', '1005105', '1004249', '1002544', '1002524', '5100816', '100042795', '5100476', '5100463', '5100577', '100084968', '10100767', '10100052', '10100054', '10100767', '10101219', '10100825', '100040048', '10400618', '7203417', '8000320', '8000040', '8000303', '8000003', '8000351', '100012123', '7900702', '7101466', '7101666', '7101898', '7101926', '7102070', '7101666', '7101819', '7101816', '7101976', '100064427', '7002254', '7001735', '7002066', '7002254', '7005191', '7003686', '7003804', '6902600']


In [26]:
%%time
# model = Word2Vec(sentences=walks, vector_size=128, window=7, workers=-1, negative=10)

model = Word2Vec(walks,  # previously generated walks
                 hs=1,  # tells the model to use hierarchical softmax
                 sg = 1,  # tells the model to use skip-gram
                 vector_size=128,  # size of the embedding
                 window=5,
                 min_count=1,
                 workers=-1,
                 seed=42)

CPU times: user 9h 11min 54s, sys: 3min 21s, total: 9h 15min 15s
Wall time: 2h 26min 56s


In [27]:
model.save('../Model/deepwalk_graph_embedding_'+GRAPH_FILE_NAME.split('.')[0]+'.model')

In [28]:
pid_set = set(graph['pid_1'].unique()).union(graph['pid_2'].unique())
payload = []
for pid in tqdm(pid_set):
    try:
        payload.append({'pid': pid, 'embedding_vector': model.wv[str(pid)]})
    except:
        print(pid, "Not Exist")
        pass

100%|██████████| 211861/211861 [00:01<00:00, 201291.01it/s]


In [29]:
embedding_df = pd.DataFrame(payload)
embedding_df.to_parquet('../Data/Embedding_Data/deepwalk_embedding_df_'+GRAPH_FILE_NAME.split('.')[0]+'.parquet', index=False)

<hr> 

## Node2vec

Setting q > 1 will favor a BFS like exploration, while setting q < 1 will favor a DFS like exploration. Hope that helps.

![Node2vec](../Documentation/Node2vec.png)

In [7]:
g = node2vec.SparseOTF(p=1, q=0.5, workers=-1, verbose=True, extend=True)

In [8]:
%%time
g.read_edg(edg_graph_path, weighted=WEIGHTED, directed=DIRECTED)

walks = g.simulate_walks(num_walks=NUM_WALK, walk_length=WALK_LEN)

Thread #  2 progress: |###                      | 9.99 %
Thread #  4 progress: |###                      | 9.99 %
Thread #  8 progress: |###                      | 9.99 %
Thread #  7 progress: |###                      | 9.99 %
Thread #  3 progress: |###                      | 9.99 %
Thread #  9Thread #  progress: |###                      | 9.99 %
 5 progress: |###                      | 9.99 %
Thread #  6 progress: |###                      | 9.99 %
Thread # 10 progress: |###                      | 9.99 %
Thread #  1 progress: |###                      | 9.99 %
Thread # 11 progress: |###                      | 9.99 %
Thread #  0 progress: |###                      | 9.99 %
Thread #  4 progress: |#####                    | 19.99 %
Thread #  2 progress: |#####                    | 19.99 %
Thread #  3 progress: |#####                    | 19.99 %
Thread #  7 progress: |#####                    | 19.99 %
Thread #  8 progress: |#####                    | 19.99 %
Thread #  5 progress: |###

In [9]:
assert len(walks) == Node_Count*NUM_WALK

In [10]:
print(walks[0])

['11500381', '11500310', '11500720', '11500159', '4804295', '5100816', '5100817', '100041309', '5100536', '5100597', '100041309', '5100818', '5100816', '5100875', '5100719', '5100564', '5100577', '100015591', '5100895', '100015591', '5100876', '5100888', '5100860', '5100852', '3601283', '3601605', '3601603', '3600661', '3601495', '3601632', '3601638', '3601015', '3600980', '3600661', '3600937', '3600666', '3600937', '3600661', '3600666', '3601347', '3600182', '3601489', '3601537', '3600937', '3601437', '3601421', '3601286', '3601454', '3600999', '3601514', '2702331']


In [11]:
%%time
model = Word2Vec(walks,  # previously generated walks
                 hs=1,  # tells the model to use hierarchical softmax
                 sg = 1,  # tells the model to use skip-gram
                 vector_size=128,  # size of the embedding
                 window=5,
                 min_count=1,
                 workers=4,
                 seed=42)

CPU times: user 4h 27min 53s, sys: 1min 1s, total: 4h 28min 55s
Wall time: 1h 9min 22s


In [12]:
model.save('../Model/node2vec_graph_embedding_'+GRAPH_FILE_NAME.split('.')[0]+'.model')

In [13]:
pid_set = set(graph['pid_1'].unique()).union(graph['pid_2'].unique())
payload = []
for pid in tqdm(pid_set):
    try:
        payload.append({'pid': pid, 'embedding_vector': model.wv[str(pid)]})
    except:
        print(pid, "Not Exist")
        pass

100%|██████████| 211861/211861 [00:01<00:00, 182278.49it/s]


In [14]:
embedding_df = pd.DataFrame(payload)
embedding_df.to_parquet('../Data/Embedding_Data/node2vec_embedding_df_'+GRAPH_FILE_NAME.split('.')[0]+'.parquet', index=False)


In [15]:
embedding_df.head()

,pid,embedding_vector
0,17301504,"[0.44851154, 0.40735474, -0.172185, 0.07374085..."
1,17301505,"[0.47892618, 0.20487863, 0.021507299, -0.62510..."
2,17301506,"[0.25423482, 0.47845665, -0.001093431, 0.03616..."
3,17301507,"[-0.10483544, 0.22859468, -0.16854355, 0.21888..."
4,17301508,"[0.83427334, 0.5198858, 0.011705197, 0.4507888..."
